## Treliça

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import solve
%matplotlib inline

In [2]:
A_0 = 100  # mm^2
E = 2e17   # N/mm^2
L = 1.5e3*np.array([np.sqrt(2),1,1])     # mm
F = np.array([0,500,-500]).reshape(3,1)  # N

In [252]:
def k_bar(Ai,Li):
    return(Ai * E / Li)

def k_matrix(A):
    A1,A2,A3 = A[0],A[1],A[2]
    L1,L2,L3 = L[0],L[1],L[2]
    k_1 = k_bar(A1,L1)
    k_2 = k_bar(A2,L2)
    k_3 = k_bar(A3,L3)
    return(np.array([k_1 + k_2,-k_1,-k_1,
                    -k_1,k_1 + k_2,k_1,
                    -k_1,k_1,k_1]).reshape(3,3))

def u_desl(A):
    K = k_matrix(A)
    u = solve(K,F)
    return(u)

def dk_dA(A):
    L1,L2,L3 = L[0],L[1],L[2]
    dk1 = E / (2 * L1) * np.array([1,-1,-1,
                                 -1,1,1,
                                 -1,1,1]).reshape(3,3)
    dk2 = E / (L2) * np.array([0,0,0,
                               0,1,0,
                               0,0,0]).reshape(3,3)
    dk3 = E / (L3) * np.array([1,0,0,
                               0,0,0,
                               0,0,0]).reshape(3,3)
    return(dk1,dk2,dk3)
    

def du_dA(A):
    u = u_desl(A)
    K = k_matrix(A)
    dk1,dk2,dk3 = dk_dA(A)
    return(solve(K,np.matmul(-dk1,u)),
           solve(K,np.matmul(-dk2,u)),
           solve(K,np.matmul(-dk3,u)))

def f(A):
    u = u_desl(A)
    return(0.5*np.matmul(F.T,u)[0])

def g_V(A):
    return(np.dot(A,L) - 300 * np.sum(L))

def g_A(A):
    return(A_0 - np.array(A))

def lagr(A):
    return(f(A) + r_v * max(0,g_V(A))**2
           + np.dot(r_A,np.maximum(g_A(A),0)**2))

def grad_dir(A):    
    L1,L2,L3 = L[0],L[1],L[2]
    Vg = r_v * max(0,g_V(A))
    Ag = r_A*np.maximum(g_A(A),0)
    uA = du_dA(A)
    return(np.array([L1 * Vg - Ag[0] +
           0.5*np.matmul(F.T,uA[0])[0][0],
           L2 * Vg - Ag[1] +
           0.5*np.matmul(F.T,uA[1])[0][0],
           L3 * Vg - Ag[2] +
           0.5*np.matmul(F.T,uA[2])[0][0]]).reshape(3,1))

def grad_adj(A):
    L1,L2,L3 = L[0],L[1],L[2]
    Vg = r_v * max(0,g_V(A))
    Ag = r_A*np.maximum(g_A(A),0)
    dk1,dk2,dk3 = dk_dA(A)
    u = u_desl(A)
    ga1 = -np.matmul(np.matmul(dk1,u).T,u)[0][0]
    ga2 = -np.matmul(np.matmul(dk2,u).T,u)[0][0]
    ga3 = -np.matmul(np.matmul(dk3,u).T,u)[0][0]
    return(np.array([L1 * Vg - Ag[0] + ga1,
                     L2 * Vg - Ag[1] + ga2,
                     L3 * Vg - Ag[2] + ga3]).reshape(3,1))

def hess_num(x0,df,alpha,d,H=np.eye(3)):
    try:
        solve(H,-df,check_finite=True).T
    except:
        H = np.eye(3)
    c = grad_adj(x0 + alpha * d)
    y = c - df
    s = alpha * d
    #print(df,d,'ys')
    if y.all() == 0:
        D = 0
    else:
        D = np.matmul(y,y.T) / np.dot(y.T,s)
    E = np.matmul(df,df.T) / np.dot(df.T[0],d)
    
    return(H + D + E)

def direction(x0,flag,alpha=1,d2f=0):
    df = grad_adj(x0)
    if flag == 0:
        d = -df.T
    elif flag == 1:
        df = grad_adj(x0)
        if type(d2f) == int:
            d2f = np.eye(3)
        try:
            d = solve(d2f,-df,sym_pos=True,check_finite=True).T[0]
        except:
            ll = 1e-1
            d2f += ll * np.eye(3)
            d = solve(d2f,-df).T[0]
            xp = x0 + alpha*d
            while lagr(x0) < lagr(xp):
                ll = ll * 10
                #print(d2f)
                if not(np.isfinite(d2f).any()):
                    d2f = ll * np.eye(3)
                else:
                    d2f = d2f + ll * np.eye(3)
                d = solve(d2f,-df).T[0]                
                xp = x0 + alpha*d
    return(d)

def step(x0,d,inc=1e-6):
    soma = 0
    f1 = lagr(x0)
    m = (np.sqrt(5) + 1) / 2
    x1 = x0 + inc*d
    while lagr(x1) > f1:
        inc = inc / 10
        x1 = x0 + inc * d
#     print(inc,'x')
    for i in range(100):
        soma += m**i
        if lagr(x0+inc*(soma)*d) > f1 and i > 0:
            break
        elif lagr(x0+inc*(soma)*d) > f1 and i == 0:
            inc = inc / 10
    I = inc * m**(i-1)*(1+m)
    T = 1 / m
    Au = inc * (soma)
    Al = inc * (soma - m**i*(1+1/m))
    Ab = T * I + Al
    Aa = (1-T) * I + Al
    fa = lagr(x0+Aa*d)
    fb = lagr(x0+Ab*d)
    while I >= 1e-1:
        if fa < fb:
            Au, Ab = Ab, Aa    
            Aa = Al + (1-T) * (Au-Al)
            fb, fa = fa, lagr(x0+Aa*d)
        elif fa > fb:
            Al, Aa = Aa, Ab            
            Ab = T * (Au-Al) + Al
            fa, fb = fb, lagr(x0+Ab*d)
        else:
            Al,Au = Aa, Ab            
            Ab = T * (Au-Al) + Al
            Aa = (1-T) * (Au-Al) + Al
            fa = lagr(x0+Aa*d)
            fb = lagr(x0+Ab*d)
        I = Au - Al
    AA = 0.5*(Au+Al)
    return(AA)

In [257]:
r_v = 1e6
r_A = [r_v]*3
A0 = [150,150,150]
l,dl= lagr(A0),grad_dir(A0)
d2l = hess_num(A0,dl,1,-dl.T[0])
print(l,np.linalg.norm(dl),d2l)
d = direction(A0,1,1,d2l)[0]
inc = 1e3
a = step(A0,d,inc)
var = A0
Appp = [[0,0,0]]*10
# print(lagr(var+a*d))

for i in range(1000):    
    if np.linalg.norm(dl) <= 1e-18:
        break
    elif i > 900 and (Appp[i] == Appp[i-20]).all():
         break
    elif lagr(var + a * d) < l:
        print(i,var)
        while lagr(var + a * d) < l:
            dA = a * d
            var += dA
            l,dl = lagr(var),grad_dir(var)
            a = step(var,d,inc)
    else:
        d = direction(var,1,a,d2l)
        d2l = hess_num(var,dl,a,d,d2l)
        a = step(var,d,inc)
        l,dl = lagr(var),grad_dir(var)
    Appp.append(var)
    
print(var,'\n',lagr(var),'\n',np.linalg.norm(dl),'\n',i)

[  4.00888348e-11] 1.74304172195e-13 [[ 0.94285714 -0.32324881 -0.0808122 ]
 [-0.32324881 -0.82857143 -0.45714286]
 [-0.0808122  -0.45714286  0.88571429]]
0 [150, 150, 150]


KeyboardInterrupt: 

In [251]:
var,i

(array([ 299.98628098,  300.0242523 ,  299.99514934]), 13)

In [150]:
g_V(var),var,f(var),g_A(var),lagr(var),np.linalg.norm(dl),a*d,d2l

(0.0,
 array([ 270.11844635,  363.80711875,  278.45177969]),
 array([  1.77928852e-11]),
 array([-170.11844635, -263.80711875, -178.45177969]),
 array([  1.77928852e-11]),
 3.0585336057277214e-14,
 array([  2.09785236e-14,   6.54208242e-14,   1.63552061e-14]),
 array([[ -81.3817363 , -257.04973958,  -64.26243489],
        [-257.04973958, -801.42154498, -200.60538624],
        [ -64.26243489, -200.60538624,  -49.15134656]]))

In [72]:
e1 = hess_num(A0,grad_dir(A0),1,-grad_dir(A0).T[0])
t1 = direction(A0,1,1,e1)[0]
a3 = step(A0,t1,1)
var = A0 + a3*t1
var

array([ 227.19582238,  485.36852431,  217.59213108])

In [70]:
e2 = hess_num(var,grad_dir(var),a3,t1)
t2 = direction(var,1,a3,e2)[0]
var += step(var,t2,1e6)*t2
var

array([ 227.19582238,  485.36852431,  217.59213108])

In [239]:
solve(d2l,-dl)

array([[ -1.10695482e-15],
       [ -6.26188205e-15],
       [ -1.56547051e-15]])